In [17]:
import os

import torch
import xarray as xr

BASE_DIR = "/research/hutchinson/data/ml_climate/saved_samples/"
CHKPT_PATH = "../checkpoints/ipsl_tas_rcp85_2.pt"
VAR = "tas"
ESM = "IPSL"
SCENARIO = "rcp85"




In [21]:
val_file = f"gen_multi_samples_{VAR}_2100-2100.nc"

# Open up the datasets
val_set = xr.open_dataset(os.path.join(BASE_DIR, ESM, SCENARIO, val_file))[VAR]

val_set

<xarray.DataArray 'tas' (time: 140, lat: 96, lon: 96)>
[1290240 values with dtype=float32]
Coordinates:
  * time     (time) object 2100-01-01 12:00:00 ... 2100-01-28 12:00:00
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2

In [16]:
cond_map = val_set.isel(time=0)
cond_map

<xarray.DataArray 'tas' (lat: 96, lon: 96)>
[9216 values with dtype=float32]
Coordinates:
    time     object 2080-01-01 12:00:00
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2

In [20]:
chkpt = torch.load(CHKPT_PATH, map_location="cpu")
model = chkpt["EMA"].to(0)
model.eval()

UNetModel3D(
  (input_conv): Conv3d(2, 64, kernel_size=(1, 7, 7), stride=(1, 1, 1), padding=(0, 3, 3))
  (time_rel_pos_bias): RelativePositionBias(
    (relative_attention_bias): Embedding(32, 8)
  )
  (input_temp_op): Residual(
    (fn): PreNorm(
      (fn): TemporalCNN(
        (temporal_conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (norm): LayerNorm()
    )
  )
  (time_mlp): Sequential(
    (0): SinusoidalPosEmb()
    (1): Linear(in_features=64, out_features=256, bias=True)
    (2): SiLU()
    (3): Linear(in_features=256, out_features=256, bias=True)
  )
  (downs): ModuleList(
    (0): ModuleList(
      (0-1): 2 x ResnetBlock(
        (mlp): Sequential(
          (0): SiLU()
          (1): Linear(in_features=256, out_features=128, bias=True)
        )
        (block1): Block(
          (proj): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
          (norm): GroupNorm(8, 64, eps=1e-05, affine=True)
          (act): SiLU()